# RNN으로_금융앱리뷰_분석하기

In [27]:
!pip install konlpy 

In [28]:
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

In [29]:
train_data = pd.read_csv("./data/bank_app_reviews_train.csv")

In [30]:
test_data = pd.read_csv("./data/bank_app_reviews_test.csv")

In [31]:
# !pip install mecab-python3

In [32]:
import re
def clean_text(text):
    cleaned = re.sub(r'[^가-힣0-9a-zA-Z\s]', '', text)  # 한글, 숫자, 영문, 공백 제외한 모든 것 제거
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned.strip()


In [51]:
train_data['is_good'] = train_data['평점'].apply(lambda x: 1 if x in [5, 4] else 0)
train_data

,리뷰일,평점,사용자리뷰,업체답변,은행명,is_good
0,2023-12-21,5,엄빠 폰에 설치해드렸는데 두분 다 쓰기 편하다고 하시네요 저도 쓰고있음,NaN,하나,1
1,2025-02-17,1,Cd기 축소 연장하려면 이 어플 깔라는데 왜 30퍼에서 안깔리는지 아니 애초에 슈퍼...,안녕하세요. 신한은행입니다.\n먼저 SOL사용에 불편을 드려 죄송합니다.\n어플의 ...,신한,0
2,2024-07-26,5,서비스가 통합되어 있어서 점점 사용빈도가 높아지네요,고객님 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사드립니다. KB스타뱅킹...,국민,1
3,2024-09-01,1,카드 충천이 안됌,"안녕하세요. 전민구 님, 토스팀입니다. 남겨주신 내용만으로는 겪고계신 불편사항의 자...",토스,0
4,2023-11-13,1,알뜰폰 인증이 안돼요,"안녕하세요 헤이모두들안녕님, 우리은행입니다. 먼저 이용에 불편을 드려 매우 죄송합니...",우리,0
...,...,...,...,...,...,...
22241,2025-03-12,3,어플 시작오류가 가끔 나네요,안녕하세요 푸딩 고객님. 우리은행 우리WON뱅킹 이용에 불편을 드려 죄송합니다. 2...,우리,0
22242,2025-01-09,1,업데이트 후에 계좌이체하는 게 자주 튕겨요,안녕하세요 일월구일 고객님. 먼저 우리WON뱅킹 이용에 불편을 드려 죄송합니다. 보...,우리,0
22243,2023-07-07,5,굳굳굳,"Seung Woo Kim 고객님, 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로...",국민,1
22244,2024-10-10,5,오래시간 같이한 나의은행 든든합니다,안녕하세요 lee jeong seon (isate) 고객님. 칭찬 진심으로 감사드리...,우리,1


In [50]:
test_data['is_good'] = test_data['평점'].apply(lambda x: 1 if x in [5, 4] else 0)
test_data

,리뷰일,평점,사용자리뷰,업체답변,은행명,is_good
0,2024-02-08,5,고경민계장님감사해요,"안녕하세요 최순녀 고객님. 칭찬 진심으로 감사드리며, 더욱 편리하고 안정적인 서비스...",우리,1
1,2023-07-24,5,저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...,"신아​ 님, 안녕하세요? 뱅크샐러드 고객감동팀​입니다. 소중한 시간내어 고객센터에 ...",뱅크샐러드,1
2,2023-09-25,1,아니 이딴걸 편리하게 사용하는앱이라고 쳐만들엇나 이렇게 불편하게만든건 일부러그런거에요,안녕하세요. 우리은행입니다. 먼저 우리WON뱅킹 이용에 불편을 드려 죄송합니다. 보...,우리,0
3,2024-02-15,3,몇 년째 만족하며 사용중이라 조금식 개선되어거는 모습에 만족하며 사용중입니다 하지만...,안녕하세요? 뱅크샐러드 고객감동팀입니다. 뱅크샐러드에 KB pay를 연결해 모든 자...,뱅크샐러드,0
4,2023-06-19,5,스타뱅킹을 사용 하고나서부터 편안해서 좋아요,"한송림 고객님, 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사드립니다. 앞...",국민,1
...,...,...,...,...,...,...
9529,2025-04-05,1,만보기 이벤트는 실망스러워요 후기 말투 다 똑같고 사기 맞죠 양심이 참 정직하게 확...,"안녕하세요. 송송님, 토스팀입니다. 만족스러운 서비스를 제공하기 위해 노력하였으나,...",토스,0
9530,2023-05-17,5,기능이 많아 다 사용해보진 못 했지만 대체적으로 편한거 같아요,이강욱 고객님 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사드립니다. 고객...,국민,1
9531,2023-07-05,5,편리하네요,"농사꾼 고객님, 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사드립니다. 앞...",국민,1
9532,2024-12-20,5,사용하기 편리해요,고객님 안녕하세요! KB스타뱅킹을 이용해 주셔서 진심으로 감사드립니다. KB스타뱅킹...,국민,1


In [33]:
train_data['사용자리뷰'] = train_data['사용자리뷰'].apply(clean_text)
train_data['사용자리뷰']

0                  엄빠 폰에 설치해드렸는데 두분 다 쓰기 편하다고 하시네요 저도 쓰고있음
1        Cd기 축소 연장하려면 이 어플 깔라는데 왜 30퍼에서 안깔리는지 아니 애초에 슈퍼...
2                             서비스가 통합되어 있어서 점점 사용빈도가 높아지네요
3                                                카드 충천이 안됌
4                                              알뜰폰 인증이 안돼요
                               ...                        
22241                                      어플 시작오류가 가끔 나네요
22242                              업데이트 후에 계좌이체하는 게 자주 튕겨요
22243                                                  굳굳굳
22244                                  오래시간 같이한 나의은행 든든합니다
22245    라이브 광고 전보다 더 안좋네 아무리 주기 싫어도 겨우 3원줄거면서 너무 쪼잔하게 ...
Name: 사용자리뷰, Length: 22246, dtype: object

In [34]:
test_data['사용자리뷰'] = test_data['사용자리뷰'].apply(clean_text)
test_data['사용자리뷰']

0                                              고경민계장님감사해요
1       저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...
2         아니 이딴걸 편리하게 사용하는앱이라고 쳐만들엇나 이렇게 불편하게만든건 일부러그런거에요
3       몇 년째 만족하며 사용중이라 조금식 개선되어거는 모습에 만족하며 사용중입니다 하지만...
4                                스타뱅킹을 사용 하고나서부터 편안해서 좋아요
                              ...                        
9529    만보기 이벤트는 실망스러워요 후기 말투 다 똑같고 사기 맞죠 양심이 참 정직하게 확...
9530                   기능이 많아 다 사용해보진 못 했지만 대체적으로 편한거 같아요
9531                                                편리하네요
9532                                            사용하기 편리해요
9533    너무 후져서 오랜만에 어플이용하다가 욕했답니다 인증방식이 2010년대에 머물러계심 ...
Name: 사용자리뷰, Length: 9534, dtype: object

토큰화

In [35]:
mecab = Mecab()
mecab.morphs(train_data['사용자리뷰'][0])

['엄',
 '빠',
 '폰',
 '에',
 '설치',
 '해',
 '드렸',
 '는데',
 '두',
 '분',
 '다',
 '쓰',
 '기',
 '편하',
 '다고',
 '하',
 '시',
 '네요',
 '저',
 '도',
 '쓰',
 '고',
 '있',
 '음']

In [36]:
tokenized_docs = train_data['사용자리뷰'].apply(mecab.morphs)

단어 인덱스 생성

In [37]:
token = Tokenizer(lower=False)
token.fit_on_texts(tokenized_docs)
print(len(token.word_index))

13010


In [38]:
x = token.texts_to_sequences(tokenized_docs)
print(x)

[[5400, 1682, 170, 11, 159, 37, 2292, 14, 301, 184, 28, 45, 19, 106, 123, 1, 76, 15, 216, 5, 45, 2, 10, 64], [7174, 19, 2404, 1948, 1, 414, 3, 18, 52, 210, 1258, 51, 521, 1604, 42, 6, 3601, 143, 90, 1518, 11, 725, 553, 1003, 114, 3, 18, 52, 435, 2, 725, 923, 11, 94, 221, 18, 29, 485, 51, 4003, 4523, 4004, 58, 277, 77, 725, 923, 923, 251, 28, 210, 13, 366, 4, 39, 250, 2830, 332], [223, 7, 815, 9, 18, 10, 88, 350, 21, 4005, 7, 797, 81, 24, 15], [77, 2655, 3, 6, 1519], [1949, 170, 47, 3, 6, 337], [30, 497, 893, 2656, 42, 2657, 5401, 59, 46, 91, 3016, 1, 13, 423, 19, 464, 2, 1, 4, 2526, 10, 14, 289, 5, 6, 9, 2, 7175, 4524, 625, 684, 5, 511, 13, 9, 18, 10, 2, 2526, 1210, 5, 6, 9, 18, 1185, 86, 30, 497, 16, 28, 48, 3262, 60], [424, 591, 19, 654, 1730, 747, 558, 108, 212, 26, 39, 668], [3263, 13, 36, 107, 100, 768], [166, 7, 30, 55, 11, 121, 3, 67, 332, 81, 10, 4, 627, 42, 373, 3, 4525, 49, 373, 12, 516, 267, 43, 14, 30, 27, 74, 210, 81, 46, 91, 894, 55, 27, 101, 85, 1, 14, 224, 112, 9, 98], 

In [39]:
max_length = max([len(i) for i in x])+1
max_length

302

가장 긴 길이에 맞춰서 패딩
* RNN의 경우 패딩을 post로 주는 것이 더 좋음. 단어가 앞, 0이 뒤에 붙는 
* Transformer계열은 위치 정보가 따로 있으므로 post, pre든 차이가 없음

In [42]:
X_padded = pad_sequences(x, maxlen=max_length, padding='post')
print(X_padded[1])

[7174   19 2404 1948    1  414    3   18   52  210 1258   51  521 1604
   42    6 3601  143   90 1518   11  725  553 1003  114    3   18   52
  435    2  725  923   11   94  221   18   29  485   51 4003 4523 4004
   58  277   77  725  923  923  251   28  210   13  366    4   39  250
 2830  332    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [43]:
X_padded.shape

(22246, 302)

In [59]:
word_size = X_padded.shape[0]

In [ ]:
y = train_data['is_good']
t.value_counts()

홀드아웃

In [46]:
# ! pip install scikit-learn

In [47]:
from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(train_data['사용자리뷰'] ,train_data['is_good'], test_size=0.4, stratify=y, random_state=42)

NameError: name 'X' is not defined

In [62]:
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, SimpleRNN, Dropout,Bidirectional 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [64]:
birnn = Sequential()
birnn.add(Input(shape=(max_length, )))
birnn.add(Embedding(input_dim=word_size, output_dim=64))
birnn.add(Bidirectional(SimpleRNN(128, return_sequences=False, activation='tanh')))
birnn.add(Dense(32, activation='relu'))
birnn.add(Dropout(0.5))
birnn.add(Dense(1, activation='sigmoid'))
birnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 302, 64)           1423744   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              49408     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,481,409
Trainable params: 1,481,409
Non-trainable params: 0
____________________________________________

In [ ]:
birnn.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy', tf.keras.metricsAUC(name='auc')])
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
model_path =  "./model/bank_app_review_birnn.keras"
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                            save_best_only=True,
                            verboss=1)
birnn_history = birnn.fit(X_train, y_train, epochs=1000, batch_size=256,
                         validation_data = (X_train, y_train),
                         callbacks= [early_stop, ])

# LSTM과 CNN 조합 모델로 분석

In [67]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D

In [ ]:
lstm_cnn = Sequential()
lstm_cnn.add(Input(shape=(max_length,)))
lstm_cnn.add(Embedding(input_dim=word_size, aoutput_dim=128))
lstm_cnn.add(Dropout(0.3))
lstm_cnn.add(Conv1D(128,5 padding='valid', activation='relu'))
lstm_cnn.add(MaxPooling1d(pool_size=4))
lstm_cnn.add(Conv1D(128,5 padding='valid', activation='relu'))
lstm_cnn.add(MaxPooling1d(pool_size=4))
lstm_cnn.add(Bidirectional(LSTM(256, return_sequences=True)))
lstm_cnn.add(GlobalMaxPooling1D())
lstm_cnn.add(Dense(64, activation='relu'))
lstm_cnn.add(Dropout(0.3))
lstm_cnn.add(Dense(32, activation='relu'))
lstm_cnn.add(Dense(1, activation='sigmoid'))
lstm_cnn.summary()

In [ ]:
birnn.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy', tf.keras.metricsAUC(name='auc')])
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
model_path =  "./model/bank_app_review_birnn.keras"
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                            save_best_only=True,
                            verboss=1)
birnn_history = birnn.fit(X_train, y_train, epochs=1000, batch_size=256,
                         validation_data = (X_train, y_train),
                         callbacks= [early_stop, ])